In [1]:
# Check GPU availability
import subprocess
import sys
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout if result.returncode == 0 else 'GPU not available')

Sun Sep 28 06:47:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import subprocess
import sys
import os
import shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Hard reset any prior torch stacks
for pkg in ('torch', 'torchvision', 'torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Create a constraints file
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# 3) Install NON-torch deps
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity check
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 537.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 339.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 377.0 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 549.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 241.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 KB 432.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 KB 439.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 532.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 533.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 508.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 534.6 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


In [5]:
import json

# Load and inspect train metadata structure
with open('train_metadata.json', 'r') as f:
    train_data = json.load(f)

print(f'train_data type: {type(train_data)}')
print(f'train_data keys: {list(train_data.keys())}')

if 'images' in train_data:
    print(f'Number of images: {len(train_data["images"])}')
    print('Sample image:', json.dumps(train_data['images'][0], indent=2))

if 'annotations' in train_data:
    print(f'Number of annotations: {len(train_data["annotations"])}')
    print('Sample annotation:', json.dumps(train_data['annotations'][0], indent=2))

if 'categories' in train_data:
    print(f'Number of categories: {len(train_data["categories"])}')
    print('Sample category:', json.dumps(train_data['categories'][0], indent=2))

train_data type: <class 'dict'>
train_data keys: ['annotations', 'categories', 'distances', 'genera', 'images', 'institutions', 'license']
Number of images: 665720
Sample image: {
  "file_name": "000/00/00000__001.jpg",
  "image_id": "00000__001",
  "license": 0
}
Number of annotations: 665720
Sample annotation: {
  "category_id": 0,
  "genus_id": 1,
  "image_id": "00000__001",
  "institution_id": 52
}
Number of categories: 15501
Sample category: {
  "authors": "(Douglas ex Loudon) J.Forbes",
  "category_id": 0,
  "family": "Pinaceae",
  "genus": "Abies",
  "scientificName": "Abies amabilis (Douglas ex Loudon) J.Forbes",
  "species": "amabilis"
}


In [7]:
import json
import pandas as pd
import numpy as np
from collections import Counter

# Load train metadata
with open('train_metadata.json', 'r') as f:
    train_data = json.load(f)

images_df = pd.DataFrame(train_data['images'])
annotations_df = pd.DataFrame(train_data['annotations'])
train_df = images_df.merge(annotations_df, on='image_id')

print(f'Train samples: {len(train_df)}')
print(f'Train columns: {train_df.columns.tolist()}')
print(f'Unique classes: {train_df["category_id"].nunique()}')

# Class distribution
class_counts = Counter(train_df['category_id'])
print(f'Min images per class: {min(class_counts.values())}')
print(f'Max images per class: {max(class_counts.values())}')
print(f'Mean images per class: {np.mean(list(class_counts.values())):.2f}')

# Top 10 classes
top_classes = class_counts.most_common(10)
print('Top 10 classes:', top_classes)

# Load test metadata
with open('test_metadata.json', 'r') as f:
    test_data = json.load(f)

test_df = pd.DataFrame(test_data)
print(f'Test samples: {len(test_df)}')
print(f'Test columns: {test_df.columns.tolist()}')

Train samples: 665720
Train columns: ['file_name', 'image_id', 'license', 'category_id', 'genus_id', 'institution_id']
Unique classes: 15501
Min images per class: 4
Max images per class: 64
Mean images per class: 42.95
Top 10 classes: [(19, 64), (123, 64), (207, 64), (230, 64), (231, 64), (297, 64), (364, 64), (572, 64), (573, 64), (578, 64)]


Test samples: 174052
Test columns: ['file_name', 'image_id', 'license']


## Initial Plan for Herbarium 2022 Competition

### Dataset Summary
- Train: 665,720 images, 15,501 classes (long-tail: 4-64 images/class, mean ~43)
- Test: 174,052 images
- Metric: Macro F1-score (emphasizes balanced performance across classes)
- Goal: Gold medal ≥0.846, Silver ≥0.754, Bronze ≥0.597

### Key Challenges
- Extreme class imbalance: Many rare classes with few images
- Large dataset: Need efficient training with GPU
- Image classification: Botanical images may require specific augmentations (e.g., rotations for leaves/flowers)

### Strategy Outline
1. **Cross-Validation**: Use StratifiedKFold(n_splits=5) to ensure balanced class representation in folds. Save fold indices to disk for reproducibility.
2. **Baseline Model**: EfficientNet-B3 from timm library (pre-trained on ImageNet). Use PyTorch for GPU acceleration.
3. **Loss Function**: Cross-entropy with class weights (inverse frequency) to handle imbalance. Consider focal loss later for better rare class handling.
4. **Data Loading & Augmentation**:
   - Use PyTorch DataLoader with num_workers=8.
   - Augmentations via Albumentations: RandomResizedCrop, HorizontalFlip, VerticalFlip, Rotate, ColorJitter, HueSaturationValue.
   - Image size: 300x300 (EfficientNet-B3 input).
   - Subsample for quick baselines: Stratified sample ~50k images (ensuring min 1 per class).
5. **Training Setup**:
   - Optimizer: AdamW, lr=1e-3, weight_decay=1e-4.
   - Scheduler: CosineAnnealingLR.
   - Early stopping on val macro F1.
   - Batch size: 32-64 depending on GPU memory.
   - Train on full dataset after baseline validation.
6. **Evaluation**: Compute macro F1 on OOF predictions. Use sklearn.metrics.f1_score(average='macro').
7. **Advanced Steps (post-baseline)**:
   - Ensemble: Blend EfficientNet variants (B0, B3, B5) + ResNet50.
   - Test-Time Augmentation (TTA).
   - Pseudo-labeling on high-confidence test predictions.
   - Feature engineering: Use genus_id or institution_id as additional features if applicable.

### Next Immediate Steps
- Install additional packages: timm, albumentations, scikit-learn (already partially installed).
- Create '01_baseline_modeling.ipynb' for data loading, CV setup, and training a quick subsample baseline.
- Request expert review on this plan, focusing on medal-winning strategies for imbalanced botanical image classification (e.g., specific augmentations, loss functions, ensemble techniques used in top solutions).

In [8]:
# Install additional packages for modeling
import subprocess
import sys
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Install timm, albumentations, opencv-python-headless
pip('install', '-c', 'constraints.txt',
    'timm', 'albumentations', 'opencv-python-headless',
    '--upgrade-strategy', 'only-if-needed')

> install -c constraints.txt timm albumentations opencv-python-headless --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 KB 356.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 309.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 420.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 KB 514.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 KB 509.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 401.5 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 201.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 265.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 457.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 449.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 455.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 372.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 367.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


In [9]:
# Reinstall CUDA-enabled PyTorch (previous install used CPU version)
import subprocess
import sys

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall current torch packages
pip('uninstall', '-y', 'torch', 'torchvision', 'torchaudio')

# Install CUDA 12.1 versions
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1+cu121', 'torchvision==0.19.1+cu121', 'torchaudio==2.4.1+cu121')

# Update constraints.txt with +cu121
from pathlib import Path
Path('constraints.txt').write_text(
    'torch==2.4.1+cu121\n'
    'torchvision==0.19.1+cu121\n'
    'torchaudio==2.4.1+cu121\n'
)

# Sanity check
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

> uninstall -y torch torchvision torchaudio


Found existing installation: torch 2.4.1+cu121


Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121
Found existing installation: torchvision 0.19.1+cu121
Uninstalling torchvision-0.19.1+cu121:
  Successfully uninstalled torchvision-0.19.1+cu121


Found existing installation: torchaudio 2.4.1+cu121
Uninstalling torchaudio-2.4.1+cu121:
  Successfully uninstalled torchaudio-2.4.1+cu121
> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 507.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 533.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 516.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
